# Training Model: SVM

## 1. Importing libraries ...

In [1]:
import pandas as pd
import numpy as np
import setup_jwlab
from jwlab.constants import cleaned_data_filepath
from jwlab.ml_prep import prep_ml, prep_ml_first20, save_ml_df, prep_ml_raw, average_trials_new, average_trials_and_participants_new
from jwlab.ml_prep_multigroup import prep_ml_multigroup
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score
from jwlab.constants import word_list

## 2. Importing Participants..

In [2]:
participants = ["904", "905"]
#participants = ["904", "905", "906", "909", "910", "912", "908", "913", "914", "916", "917", "919", "920", "921", "923", "924","927", "928", "929", "930", "932"]

#9m with >40 trials
#participants = [ "909", "912", "908", "913", "914", "916", "917", "919", "920", "921", "924","927", "930"]

#12m all
#participants = [ "107", "109", "111", "112", "115", "116", "117", "119", "121", "122", "120", "124"]

#12m with >40 trials
#participants = ["109", "111", "112", "115", "124"]

#all participants
# participants = ["904", "905", "906", "909", "910", "912", "908", "913", "914", "916", "917", "919", "920", "921", "923", "924","927", "928", "929", "930", "932",
#               "107", "109", "111", "112", "115", "116", "117", "119", "121", "122", "120", "124"]



## 3. Set up averaging

In [3]:
X, y, good_trial_count = prep_ml_raw(cleaned_data_filepath, participants, downsample_num=1000, averaging="no_averaging")


loaded


In [4]:
#Randomize and split
X[0][0] = X[0][0].sample(frac=1).reset_index(drop=True)
len(X[0][0])
fivefold_testsize = int(.20*len(X[0][0]))
fivefold_trainsize = len(X[0][0])- fivefold_testsize
df_train = X[0][0][:fivefold_trainsize]
df_test = X[0][0][:fivefold_testsize]


[             0          1          2          3          4          5  \
 0     1.568287   0.258826   2.369213   0.539767  -1.858165  -0.647701   
 1     1.253755   2.377808   5.996650   4.433384   6.518006   4.154214   
 2    -1.043779  -8.096023  -8.190574  -7.923188  -7.043860  -8.094473   
 3     3.224830  -5.427824  -8.399541 -11.078480 -10.747657  -8.991525   
 4     3.966432  27.550192  25.235700  22.818636   9.502732  13.774457   
 ..         ...        ...        ...        ...        ...        ...   
 189  -6.618451  22.140377  30.444341  25.635734  25.399753  20.738769   
 190   7.294317  -6.102833 -22.045662 -25.298935 -25.680079 -17.646832   
 192 -57.880028   4.193850  16.717931  20.435879   4.981079  23.077141   
 195  20.327205   7.483005   6.199234   6.256866  -9.018277   1.643158   
 197 -28.453903 -14.257771  -2.032366   5.626649   0.949348   6.495548   
 
              6          7          8          9  ...      59992       59993  \
 0     0.846586   0.932043  10

(208, 60002)

In [5]:
#calc averages for ps
num_participants = len(participants)
num_words = len(word_list)

X_testp, y_testp, ps_p, words_p = average_trials_new(df_test, num_participants, num_words)
        
#new_data and new_y contains avg and 

In [6]:
#calc averages for ps and trial
num_participants = len(participants)
num_words = len(word_list)

X_testpt, y_testpt, ps_pt, words_pt = average_trials_and_participants_new(df_test, num_participants, num_words)
        

In [7]:
y_train = df_train['label']

X_train = df_train.drop(columns = ['label', 'participant'])

In [14]:
df_train

,0,1,2,3,4,5,6,7,8,9,...,59991,59992,59993,59994,59995,59996,59997,59998,59999,participant
0,5.025561,2.821668,1.354192,-2.142615,11.336155,-4.152444,-2.091002,-3.415384,0.031801,3.663936,...,-36.783891,-51.763780,-10.839019,-1.379785,-0.941817,2.423776,25.240828,16.091168,18.888030,1
1,1.226106,-8.968997,-7.018534,-7.623764,-15.155371,-6.685738,-1.507917,1.371341,7.471295,3.562313,...,-11.172329,-9.658216,-18.955927,-22.915535,-26.130806,-21.348746,-21.570639,-28.130672,-23.931409,1
2,-21.440239,-11.394920,6.691976,19.928586,6.939685,35.958892,25.973378,30.300085,-30.329784,-9.361191,...,-36.792358,-13.124541,-105.154893,-75.082496,-25.566505,20.426337,128.182099,-45.518128,24.477999,1
3,10.662455,15.114633,-3.685214,-10.098062,-31.630879,-9.824709,-9.289262,-1.059709,-3.211483,-16.467025,...,-15.633615,-25.227271,-13.470614,-19.928415,-5.038135,-18.635382,0.838223,-5.185655,-8.765579,1
4,13.464834,-15.767405,-15.879229,-14.033575,-13.525173,-11.415341,-9.476569,-6.431889,6.891097,-2.698541,...,40.235585,79.385104,78.691800,71.961547,-18.500822,23.234891,-19.802306,-48.267835,-5.302885,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83,1.011107,-0.425666,4.587227,11.496941,0.599010,7.097873,10.544865,11.428293,-10.432805,-14.142159,...,-12.445938,1.555545,-4.700431,-7.531514,5.121854,8.324526,24.742058,-2.644727,5.680951,1
84,9.999920,-6.363606,-3.636014,-2.824049,-3.502189,-2.205377,-0.452135,0.080261,-0.182014,-4.367211,...,-17.428820,-46.723771,51.654913,14.039964,-39.187296,-25.556754,-11.960428,-75.563740,-28.008781,0
85,-4.167489,-4.783387,-0.143864,1.803748,4.888226,1.685998,11.039969,13.169111,1.777397,4.557585,...,-8.918173,22.553037,-27.569104,-41.604561,30.247673,9.502596,108.050071,15.358838,47.472026,1
86,-2.415063,-12.980877,0.472723,6.273451,17.825636,4.640987,6.232900,6.746609,-10.626613,0.339670,...,8.698651,4.078425,10.629541,24.343760,-14.279415,-4.556163,6.566625,-11.057235,0.446281,1


In [6]:
X_testpt.shape

(16, 60000)

In [4]:
good_trial_count

[array([0.45, 0.69, 0.53, 0.67, 0.25, 0.55, 0.76, 0.86, 0.7 , 0.75, 0.94,
        0.79, 0.67, 0.8 , 0.73, 0.71, 0.8 , 0.5 , 0.57, 0.87, 0.63]), [cell
  1    15
  2    13
  3    12
  4    13
  dtype: int64, cell
  1    16
  2    17
  3    15
  4    10
  dtype: int64, cell
  1     9
  2     7
  3     8
  4    10
  dtype: int64, 1     0
  2     2
  3     1
  4     1
  5     1
  6     2
  7     2
  8     1
  9     2
  10    2
  11    1
  12    1
  13    1
  14    2
  15    1
  16    0
  17    1
  18    1
  19    1
  20    1
  21    1
  22    1
  23    2
  24    2
  25    2
  26    1
  27    1
  28    2
  29    2
  30    2
  31    1
  32    1
  dtype: int64, 1     1
  2     1
  3     0
  4     0
  5     1
  6     0
  7     1
  8     1
  9     1
  10    0
  11    0
  12    0
  13    0
  14    0
  15    0
  16    1
  17    0
  18    1
  19    1
  20    0
  21    1
  22    0
  23    1
  24    1
  25    1
  26    0
  27    0
  28    0
  29    0
  30    0
  31    0
  32    0
  dtype: int64, 1   

In [4]:
y

[[array([0., 1., 0., 1., 0., 1., 0., 1., 1., 0., 1., 0., 1., 0., 0., 1., 0.,
         1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 1., 0., 1., 0., 0., 1.,
         0., 1., 0., 1., 0., 1., 0., 0., 0., 0., 1., 1., 0., 1., 0., 0., 1.,
         0.])]]

In [5]:
#save_ml_df(X[0][0], "/Users/JennMacBook/Desktop/Studies/Animates EEG/7_Data/runningOffline/june17data12m.pkl")
np.savetxt('y12m.out', y[0][0])

In [ ]:
good_trial_count

## 3. b) Different groups

In [ ]:
X, y, good_trial_count = prep_ml_multigroup(cleaned_data_filepath, participants, downsample_num=1000, averaging="no_averaging")
Xt, yt, good_trial_count_t = prep_ml_multigroup(cleaned_data_filepath, participants, downsample_num=1000, averaging="average_trials")
Xa, ya, good_trial_count_a = prep_ml_multigroup(cleaned_data_filepath, participants, downsample_num=1000, averaging="average_trials_and_participants")


In [ ]:
#good_trial_count

In [ ]:
(n, d) = X[0][0].shape
assert n == y[0][0].shape[0]


In [ ]:
d

In [ ]:
unique, counts = np.unique(y[0][0], return_counts=True)
dict(zip(unique, counts))

In [ ]:
(n, d) = Xt[0][0].shape
assert n == yt[0][0].shape[0]


n

In [ ]:
d

In [ ]:
unique, counts = np.unique(yt[0][0], return_counts=True)
dict(zip(unique, counts))

In [ ]:
(n, d) = Xa[0][0].shape
assert n == ya[0][0].shape[0]
n

In [ ]:
d

In [ ]:
unique, counts = np.unique(ya[0][0], return_counts=True)
dict(zip(unique, counts))

## 3. c) First 20 trials


In [ ]:
X, y, good_trial_count = prep_ml_first20(cleaned_data_filepath, participants, downsample_num=1000, averaging="no_averaging")
Xt, yt, good_trial_count_t = prep_ml_first20(cleaned_data_filepath, participants, downsample_num=1000, averaging="average_trials")
Xa, ya, good_trial_count_a = prep_ml_first20(cleaned_data_filepath, participants, downsample_num=1000, averaging="average_trials_and_participants")



In [ ]:
(n, d) = X[0][0].shape
assert n == y[0][0].shape[0]
n

In [ ]:
d

## 4. Setting up the SVM model ...

In [8]:
model = LinearSVC(C=1e-9, max_iter=5000)
#model = SVC(gamma=.001, kernel = 'rbf', C = 1e-6)

## 5. Training and testing the model ...

### 5.1. Train on raw, test on raw (validation)

In [9]:
# from sklearn import preprocessing

# scaler = preprocessing.StandardScaler().fit(X)
# X,Xp = scaler.transform(X), scaler.transform(Xp)
model.fit(X_train, y_train)
np.mean(model.predict(X_train) != y_train)

0.475

### 5.2. Train on raw, test on avg by trial (word repetition) 

In [10]:
# from sklearn import preprocessing

# scaler = preprocessing.StandardScaler().fit(X)
# X,Xp = scaler.transform(X), scaler.transform(Xp)
model.fit(X_train, y_train)
np.mean(model.predict(X_testp) != y_testp)

0.6770833333333334

### 5.3. Train on raw, test on avg by word and ps 

In [11]:
# from sklearn import preprocessing

# scaler = preprocessing.StandardScaler().fit(X)
# X,Xp = scaler.transform(X), scaler.transform(Xp)
model.fit(X_train, y_train)
np.mean(model.predict(X_testpt) != y_testpt)

0.0

# 6 Subset analysis 

## 6.1 Generating random subsets of the chosen participant list

In [ ]:
participants_train, participants_test = train_test_split(participants,test_size=0.2)
print(len(participants_train), len(participants_test))

### 6.2.1 Create train and test sets: animates/inanimates

In [ ]:
#X_train, y_train, good_trial_count_train = prep_ml(cleaned_data_filepath, participants_train, downsample_num=1000, averaging="no_averaging")
#X_test, y_test, good_trial_count_test = prep_ml(cleaned_data_filepath, participants_test, downsample_num=1000, averaging="no_averaging")

X_train, y_train, good_trial_count_train = prep_ml(cleaned_data_filepath, participants_train, downsample_num=1000, averaging="average_trials")
X_test, y_test, good_trial_count_test = prep_ml(cleaned_data_filepath, participants_test, downsample_num=1000, averaging="average_trials")

#X_train, y_train, good_trial_count_train = prep_ml(cleaned_data_filepath, participants_train, downsample_num=1000, averaging="average_trials_and_participants")
#X_test, y_test, good_trial_count_test = prep_ml(cleaned_data_filepath, participants_test, downsample_num=1000, averaging="average_trials_and_participants")

### 6.2.2 Create train and test sets - multi group

In [ ]:
#X_train, y_train, good_trial_count_train = prep_ml_multigroup(cleaned_data_filepath, participants_train, downsample_num=1000, averaging="no_averaging")
#X_test, y_test, good_trial_count_test = prep_ml_multigroup(cleaned_data_filepath, participants_test, downsample_num=1000, averaging="no_averaging")

#X_train, y_train, good_trial_count_train = prep_ml_multigroup(cleaned_data_filepath, participants_train, downsample_num=1000, averaging="average_trials")
#X_test, y_test, good_trial_count_test = prep_ml_multigroup(cleaned_data_filepath, participants_test, downsample_num=1000, averaging="average_trials")

X_train, y_train, good_trial_count_train = prep_ml_multigroup(cleaned_data_filepath, participants_train, downsample_num=1000, averaging="average_trials_and_participants")
X_test, y_test, good_trial_count_test = prep_ml_multigroup(cleaned_data_filepath, participants_test, downsample_num=1000, averaging="average_trials_and_participants")

### 6.2.3 Create train and test sets - first 20 trials

In [ ]:
#X_train, y_train, good_trial_count_train = prep_ml_first20(cleaned_data_filepath, participants_train, downsample_num=1000, averaging="no_averaging")
#X_test, y_test, good_trial_count_test = prep_ml_first20(cleaned_data_filepath, participants_test, downsample_num=1000, averaging="no_averaging")

#X_train, y_train, good_trial_count_train = prep_ml_first20(cleaned_data_filepath, participants_train, downsample_num=1000, averaging="average_trials")
#X_test, y_test, good_trial_count_test = prep_ml_first20(cleaned_data_filepath, participants_test, downsample_num=1000, averaging="average_trials")

X_train, y_train, good_trial_count_train = prep_ml_first20(cleaned_data_filepath, participants_train, downsample_num=1000, averaging="average_trials_and_participants")
X_test, y_test, good_trial_count_test = prep_ml_first20(cleaned_data_filepath, participants_test, downsample_num=1000, averaging="average_trials_and_participants")

In [ ]:
(n, d) = X_train[0][0].shape
assert n == y_test[0][0].shape[0]
n

In [ ]:
X_train[0][0].shape

### 6.2.4 Classification

In [ ]:
#model = SVC(gamma=.001, kernel = 'rbf', C=1e-6)
model = SVC(kernel = 'rbf')
model.fit(X_train[0][0], y_train[0][0])
np.mean(model.predict(X_test[0][0]) != y_test[0][0])

## 6.3 Monte Carlo Classification

In [ ]:

errorScores = []

# r iterations of a 5 fold
for r in range(2):
    kgp = np.array_split(participants, 5)

    for i in range(5):
        participants_test = kgp[i]
        participants_train = np.concatenate((kgp[(i+1)%5], kgp[(i+2)%5],kgp[(i+3)%5],kgp[(i+4)%5] ), axis=0)
        
        #X_train, y_train, good_trial_count_train = prep_ml(cleaned_data_filepath, participants_train, downsample_num=1000, averaging="no_averaging")
        #X_test, y_test, good_trial_count_test = prep_ml(cleaned_data_filepath, participants_test, downsample_num=1000, averaging="no_averaging")

        X_train, y_train, good_trial_count_train = prep_ml(cleaned_data_filepath, participants_train, downsample_num=1000, averaging="average_trials")
        X_test, y_test, good_trial_count_test = prep_ml(cleaned_data_filepath, participants_test, downsample_num=1000, averaging="average_trials")


        #X_train, y_train, good_trial_count_train = prep_ml(cleaned_data_filepath, participants_train, downsample_num=1000, averaging="average_trials_and_participants")
        #X_test, y_test, good_trial_count_test = prep_ml(cleaned_data_filepath, participants_test, downsample_num=1000, averaging="average_trials_and_participants")

    
        #model = SVC(gamma=.001, kernel = 'rbf', C=1e-06)
        model = LinearSVC(C=1e-9, max_iter=5000)
        model.fit(X_train[0][0], y_train[0][0])
        errorScore = np.mean(model.predict(X_test[0][0]) != y_test[0][0])
        errorScores.append(errorScore)
    
errorScores



errorScores

In [ ]:
np.mean(errorScores)

In [ ]:
np.std(errorScores)

## 6.4 Alternate accuracy measurements

In [ ]:
from sklearn.model_selection import train_test_split

y_pred = model.predict(X_test[0][0])

from sklearn.metrics import accuracy_score,recall_score,precision_score,f1_score
print('Accuracy Score : ' + str(accuracy_score(y_test[0][0],y_pred)))
print('Precision Score : ' + str(precision_score(y_test[0][0],y_pred)))
print('Recall Score : ' + str(recall_score(y_test[0][0],y_pred)))
print('F1 Score : ' + str(f1_score(y_test[0][0],y_pred)))

from sklearn.metrics import confusion_matrix
print('Confusion Matrix : \n' + str(confusion_matrix(y_test[0][0],y_pred)))

### 7 Optimization

In [ ]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
parameters = {'kernel': ['rbf'], 'gamma': [1e-3, 1e-4],
                     'C': [1e-6, 1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1, 10, 100, 1000]}
svc = SVC()
model = GridSearchCV(svc, parameters, verbose=True)
model.fit(X_train[0][0], y_train[0][0])

model.cv_results_

In [ ]:
model.best_score_

In [ ]:
model.best_estimator_

# Cross Validation (5 fold)

In [ ]:
Xfirst = X[0][0]
yfirst = y[0][0]
# Xfirst = Xt[0][0]
# yfirst = yt[0][0]
# Xfirst = Xa[0][0]
# yfirst = ya[0][0]

Xfirst['label'] = yfirst
Xfirst = Xfirst.sample(frac=1).reset_index(drop=True)
ys = Xfirst['label']
Xs = Xfirst.drop(columns=['label'])


In [ ]:

model = LinearSVC(max_iter=5000, C=1e-9)
#model = SVC(gamma=.001, kernel = 'rbf', C = 1e-6)

In [ ]:
scores = cross_val_score(model, X[0][0], y[0][0], cv=5)
print(scores)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

In [ ]:
scores = cross_val_score(model, Xs, ys, cv=5)
print(scores)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

# Repeated N-Folds

In [ ]:
import numpy as np
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,recall_score,precision_score,f1_score

testScores = []

X = X[0][0]
y = y[0][0]

# X = Xt[0][0]
# y = yt[0][0]

rkf = RepeatedKFold(n_splits=5, n_repeats=2, random_state=2652124)
for train_index, test_index in rkf.split(X):
    #print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    model = LinearSVC(C=1e-9, max_iter=5000)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    testScore = accuracy_score(y_test,y_pred)
    testScores.append(testScore)


testScores

In [ ]:
print(X.shape)

In [ ]:
print(X_train.shape)

In [ ]:
print(np.mean(testScores))
print(np.std(testScores))

In [ ]:
X.shape

In [ ]:
X_train.shape